In [ ]:
import pandas as pd
from openai import OpenAI
import numpy as np

# ==== CONFIGURA TU API KEY ====

client = OpenAI(api_key="")

In [2]:
# ========== 3. EJEMPLOS DEL CSV (QA) ==========
df = pd.read_csv("../Hackathon Sistecredito/Datasets/negociaciones_chatbot.csv")

In [ ]:
# Se crean los Agentes
plantillas1 = {
    "prejuridico": """Te informo que el crédito que tienes activo, se encuentra en proceso de cobro
perjudico con:
Días de mora:
Saldo total:
Por tal razón, el valor a pagar en el momento corresponde al valor capital,
financiación, intereses de mora y cargos pre jurídicos. Actualmente tienes el reporte
negativo en centrales de riesgo.""",

    "administrativo": """Te informo que, en el momento tienes los siguientes créditos activos:
Crédito:
Almacén:
Pago mínimo al día de hoy:
Días en mora:
Cuotas vencidas:
Pago total al día de hoy:
¿Para qué fecha realizarías el pago?""",

    "beneficio_eliminacion_sin_reporte": """El pago oportuno de tu compromiso te permitirá seguir disfrutando de tu crédito y
evitarás tener un reporte negativo en las centrales de riesgo.""",

    "beneficio_eliminacion_con_reporte": """El pago oportuno de tu compromiso te permitirá seguir disfrutando de tu crédito y
evitarás tener un nuevo reporte negativo en las centrales de riesgo.""",

    "confirmacion_compromiso_administrativo": """Tu compromiso fue generado de manera exitosa. Te confirmo que para el día
********, tu saldo a cancelar es:
Crédito:
Almacén:
Pago mínimo:""",

    "confirmacion_compromiso_prejuridico_cuotas": """Te confirmo que tu compromiso fue generado,
cuotas:
Frecuencia: mensuales/quincenales,
valor cuota $
Fecha de inicio
El acuerdo de pago no te exime de los reportes negativos ante las centrales de riesgo,
es importante que cumplas con las fechas pactadas para evitar la reliquidación de
los intereses. El pago oportuno de este compromiso te permitirá seguir disfrutando
de tu crédito, evitar un proceso jurídico y mejorar tu calificación crediticia.""",

    "confirmacion_compromiso_prejuridico_descuento": """Te confirmo que tu compromiso fue generado,
Valor total a pagar:
Fecha:
Descuento del XX% en cargos prejuridicos e intereses moratorios.
Es importante que cumplas con las fechas pactadas para evitar la reliquidación de
los intereses. El pago oportuno de este compromiso te permitirá seguir disfrutando
de tu crédito, evitar un proceso jurídico y mejorar tu calificación crediticia.""",

    "confirmacion_compromiso_prejuridico_total_con_beneficio": """Te confirmo el compromiso pactado.
Saldo total:
Fecha:
Con el beneficio de eliminación en centrales. Una vez realices el pago se verá
reflejada en un tiempo estimado de 12 días hábiles. Recuerda cumplir con la fecha
acordada para evitar la reliquidación de los intereses. El pago oportuno de este
compromiso te permitirá seguir disfrutando de tu crédito, evitar un proceso jurídico y
mejorar tu calificación crediticia.""",

    "negociacion_prejuridico_eliminacion": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer la eliminación de tu reporte en las centrales de riesgo; si cancelas
todo el valor en el plazo de un mes, te eliminamos los reportes de tu historial
crediticio de las obligaciones a las cuales se les va a realizar el compromiso,
teniendo en cuenta que así no cumplirías con el tiempo de permanencia.""",

    "negociacion_prejuridico_cuotas": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer un acuerdo de pago a cuotas mensuales o quincenales, el plazo
máximo serían XX cuotas, cada una por un valor de $XXXXXXX. El beneficio de este
compromiso es que se te congelan los intereses siempre y cuando cumplas con las
fechas acordadas.""",

    "negociacion_prejuridico_descuento": """Mi objetivo es ayudarte a que puedas ponerte al día con esta obligación, por eso, te
puedo ofrecer un descuento del X% sobre los cargos prejuridicos y el valor en mora;
si realizas el pago total en el plazo máximo de un mes, sería un saldo total de
$XXXXXXXX.""",

    "negociacion_refinanciacion": """Te informo que, en el momento podemos refinanciar #XXX créditos, el saldo total a
refinanciar es de $XXXX, quedaría en #XX cuotas mensuales por un valor de $XXX, la
primera cuota seria para X/X/2024. Es importante tener en cuenta que el cupo
disponible quedará en cero. ¿Deseas continuar con el proceso?"""
}

# ========== 2. EMBEDDINGS Y DETECCIÓN AUTOMÁTICA ==========
def obtener_embedding(texto, modelo="text-embedding-ada-002"):
    try:
        response = client.embeddings.create(
            model=modelo,
            input=texto
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"❌ Error generando embedding para: '{texto[:60]}...' -> {e}")
        return None

def similitud_coseno(vec1, vec2):
    if vec1 is None or vec2 is None:
        return -1  # devuelve una similitud baja si hay error
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def construir_base_embeddings_plantillas(plantillas_dict):
    base = []
    for nombre, texto in plantillas_dict.items():
        emb = obtener_embedding(texto)
        if emb is not None:
            base.append({
                "tipo": nombre,
                "embedding": emb,
                "texto": texto
            })
        else:
            print(f"⚠️ Se omitió la plantilla '{nombre}' por error en el embedding.")
    return base


def detectar_tipo_plantilla(mensaje_cliente, base_embeddings):
    emb_mensaje = obtener_embedding(mensaje_cliente)
    mejor_similitud = -1
    mejor_tipo = None

    for item in base_embeddings:
        similitud = similitud_coseno(emb_mensaje, item['embedding'])
        if similitud > mejor_similitud:
            mejor_similitud = similitud
            mejor_tipo = item['tipo']

    return mejor_tipo


# df = pd.read_csv(io.BytesIO(uploaded['negociaciones_chatbot (5).csv']))  # reemplaza con tu nombre de archivo
print(df.columns)
ejemplos = df[["Mensajes Agente", "Mensajes Cliente"]].to_dict(orient="records")


Index(['Mensajes Agente', 'Mensajes Cliente'], dtype='object')


In [ ]:
# ========== 4. CONSTRUIR PROMPT Y GENERAR RESPUESTA ==========
def construir_prompt(mensaje_usuario, tipo_plantilla, ejemplos, plantillas, num_ejemplos=2):
    prompt = "Eres un agente de cartera. Aquí tienes ejemplos de conversaciones anteriores:\n\n"
    for i in range(min(num_ejemplos, len(ejemplos))):
        ej = ejemplos[i]
        prompt += f"Ejemplo {i+1}:\nCliente: {ej.get('cliente')}\nAgente: {ej.get('agente')}\n\n"

    prompt += "---\nUsa el siguiente mensaje del cliente y responde siguiendo esta plantilla:\n"
    prompt += f"\nPLANTILLA ({tipo_plantilla.upper()}):\n{plantillas.get(tipo_plantilla, '')}\n"
    prompt += f"\nMensaje del cliente: \"{mensaje_usuario}\"\n\nRespuesta del agente:"
    return prompt

def generar_respuesta(mensaje_usuario, ejemplos, plantillas, base_embeddings):
    tipo = detectar_tipo_plantilla(mensaje_usuario, base_embeddings)
    prompt = construir_prompt(mensaje_usuario, tipo, ejemplos, plantillas)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=400
    )

    respuesta = response.choices[0].message.content
    return tipo, respuesta  # ← ¡esto es lo que faltaba!


# ========== 5. INTERFAZ DE CONSOLA ==========
def chatbot():
    print("📞 Chatbot iniciado. Escribe 'salir' para terminar.\n")

    # Cargar recursos
    plantillas = plantillas1
    base_embeddings = construir_base_embeddings_plantillas(plantillas)

    while True:
        mensaje = input("Cliente: ")
        if mensaje.lower() == "salir":
            break

        tipo_detectado, respuesta = generar_respuesta(mensaje,ejemplos, plantillas, base_embeddings)
        #print(f"\n🔎 Tipo de plantilla usada: {tipo_detectado}\n")
        print("Agente:", respuesta)
        print("\n---\n")

# Ejecutar
if __name__ == "__main__":
    chatbot()


📞 Chatbot iniciado. Escribe 'salir' para terminar.

Agente: PLANTILLA (ADMINISTRATIVO):
Te informo que, en el momento tienes los siguientes créditos activos:
Crédito:
Almacén:
Pago mínimo al día de hoy:
Días en mora:
Cuotas vencidas:
Pago total al día de hoy:
¿Para qué fecha realizarías el pago?

Mensaje del cliente: "Hola, dime cuales son mis créditos activos"

Respuesta del agente: En este momento tienes los siguientes créditos activos:
Crédito: Préstamo personal
Almacén: Sucursal 123
Pago mínimo al día de hoy: $50
Días en mora: 10 días
Cuotas vencidas: 2 cuotas
Pago total al día de hoy: $200
¿Para qué fecha realizarías el pago?

---

Agente: Lamento informarte que no puedo completar la plantilla proporcionada, ya que no tengo acceso a la información específica de tu compromiso, valor total a pagar, fecha, descuento, cargos prejurídicos e intereses moratorios. Si necesitas ayuda con algún tema relacionado con tu cartera, por favor proporciona más detalles para que pueda brindarte la 